In [ ]:
import json
import requests
import time
import sys, urllib
from bs4 import BeautifulSoup
from bs4.element import NavigableString


root = "https://www.ptt.cc/bbs/"
main = "https://www.ptt.cc"
gossip_data = {
        "from":"bbs/Gossiping/index.html",
        "yes": "yes"
    }

rs = requests.session()

#爬文章網址
def articles(page):
    requests.packages.urllib3.disable_warnings()
    res = rs.post('https://www.ptt.cc/ask/over18', verify=False, data=gossip_data)
    res  = rs.get(page)
    soup = BeautifulSoup(res.text, "lxml")

    for article in soup.select(".r-ent"):
        try:
            yield main + article.select(".title")[0].select("a")[0].get("href")
        except:
            pass # (本文已被刪除)

#轉換時間格式
def timechange(day):
    timeString = day
    struct_time = time.strptime(timeString, "%a %b %d %H:%M:%S %Y")
    new_timeString = time.strftime("%Y-%m-%d", struct_time)
    return new_timeString

#爬內文    
def parse_article(url, corp = '王品集團', Brand = "饗饗"):       
    raw  = rs.get(url)
    soup = BeautifulSoup(raw.text, "lxml")

    try:
        article = {}

        # 取得文章作者與文章標題
        article["Corp"]   = corp
        article["Brand"]  = Brand
        article["Author"] = soup.select(".article-meta-value")[0].contents[0].split(" ")[0]
        article["Title"]  = soup.select(".article-meta-value")[2].contents[0]
        article["Time"]   = timechange(soup.select(".article-meta-value")[3].contents[0])

        # 取得內文
        content = ""
        for tag in soup.select("#main-content")[0]:
            if type(tag) is NavigableString and tag !='\n':
                content += tag
                break
        article["Content"] = content

        # 處理回文資訊
        upvote = 0
        downvote = 0
        novote = 0
        response_list = []

        for response_struct in soup.select(".push"):
            #跳脫「檔案過大！部分文章無法顯示」的 push class
            if "warning-box" not in response_struct['class']:
                response_dic = {}
                
                response_dic["User"]  = response_struct.select(".push-userid")[0].contents[0]    
                response_dic["Content"] = response_struct.select(".push-content")[0].contents[0][1:]
                response_dic["Vote"]  = response_struct.select(".push-tag")[0].contents[0][0]
                response_list.append(response_dic)
                        
                if response_dic["Vote"] == u"推":
                    upvote += 1
                elif response_dic["Vote"] == u"噓":
                    downvote += 1
                else:
                    novote += 1

        article["Responses"] = response_list
        article["UpVote"] = upvote
        article["DownVote"] = downvote
        article["NoVote"] = novote

    except Exception as e:
        print(e)
        print(u"在分析 %s 時出現錯誤" % url)

    return article

def output(filename, data):
    try:
        with open(filename +".json", 'wb+') as f:
            f.write(json.dumps(data, indent=1, ensure_ascii=False).encode('utf-8'))
            print('爬取完成', filename + '.json', '輸出成功')
    except Exception as err:
        print(filename +'.json', '輸出失敗')
        print('error message:', err)

def ptt_crawler(board = 'Gossiping', corp = '王品集團', search = '%E7%8E%8B%E5%93%81', start_page = 1, end_page = 2):
    res = []
    for j in range(start_page, end_page+1):
        url = 'https://www.ptt.cc/bbs/'+ board + '/search?page='+ str(j) + '&q=' + search
        for i in articles(url):
            res.append(parse_article(i, corp = corp, Brand = search))
            time.sleep(1)   
        print(u"已經完成 %s 頁面第 %d 頁的爬取" %(board, j))

    output('ptt_'+ board+ '_'+ search, res) 

    
#饗饗 %E9%A5%97%E9%A5%97
# ptt_crawler(board = 'Gossiping', search = '%E9%A5%97%E9%A5%97', start_page = 1, end_page = 4)


In [ ]:
#Gossip
#饗饗 2頁
ptt_crawler(board = 'Gossiping', corp = '饗賓集團', search = '饗饗', start_page = 1, end_page = 2)

已經完成 Gossiping 頁面第 1 頁的爬取
已經完成 Gossiping 頁面第 2 頁的爬取
爬取完成 ptt_Gossiping_饗饗.json 輸出成功


In [ ]:
#Gossip
#饗食天堂 2頁
ptt_crawler(board = 'Gossiping', corp = '饗賓集團', search = '饗食天堂', start_page = 1, end_page = 2)

已經完成 Gossiping 頁面第 1 頁的爬取
已經完成 Gossiping 頁面第 2 頁的爬取
爬取完成 ptt_Gossiping_饗食天堂.json 輸出成功


In [ ]:
#Gossip
#旭集 2頁
ptt_crawler(board = 'Gossiping', corp = '饗賓集團', search = '旭集', start_page = 1, end_page = 2)

已經完成 Gossiping 頁面第 1 頁的爬取
已經完成 Gossiping 頁面第 2 頁的爬取
爬取完成 ptt_Gossiping_旭集.json 輸出成功


In [ ]:
#Gossip
#開飯川 1頁
ptt_crawler(board = 'Gossiping', corp = '饗賓集團', search = '開飯川', start_page = 1, end_page = 1)

已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_開飯川.json 輸出成功


In [ ]:
#Gossip
#果然匯 1頁
ptt_crawler(board = 'Gossiping', corp = '饗賓集團', search = '果然匯', start_page = 1, end_page = 1)

已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_果然匯.json 輸出成功


In [ ]:
#Food
#饗饗  1頁
ptt_crawler(board = 'Food', corp = '饗賓集團', search = '饗饗', start_page = 1, end_page = 1)

list index out of range
在分析 https://www.ptt.cc/bbs/Food/M.1569208484.A.6B1.html 時出現錯誤
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_饗饗.json 輸出成功


In [ ]:
#Food
#饗食天堂 1頁
ptt_crawler(board = 'Food', corp = '饗賓集團', search = '饗食天堂', start_page = 1, end_page = 1)

已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_饗食天堂.json 輸出成功


In [ ]:
#Food
#旭集 1頁
ptt_crawler(board = 'Food', corp = '饗賓集團', search = '旭集', start_page = 1, end_page = 1)

已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_旭集.json 輸出成功


In [ ]:
#Food
#開飯川 1頁
ptt_crawler(board = 'Food', corp = '饗賓集團', search = '開飯川', start_page = 1, end_page = 1)

已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_開飯川.json 輸出成功


In [ ]:
#Food
#果然匯 1頁
ptt_crawler(board = 'Food', corp = '饗賓集團', search = '果然匯', start_page = 1, end_page = 1)

已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_果然匯.json 輸出成功


In [ ]:
#Food
#饗泰多 1頁
ptt_crawler(board = 'Food', corp = '饗賓集團', search = '饗泰多', start_page = 1, end_page = 1)

已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_饗泰多.json 輸出成功


In [ ]:
#Gossip
#王品牛排 1頁
# ll = ['王品牛排','西堤','陶板屋','原燒','藝奇','夏慕尼','石二鍋','品田牧場','青花驕','享鴨','the wang','和牛涮','丰禾','肉次方','hot7','尬鍋','最肉','嚮辣','12mini']
ll = ['王品牛排','陶板屋','藝奇','夏慕尼','品田牧場','青花驕','享鴨','和牛涮','肉次方','12mini']
for i in ll:
    ptt_crawler(board = 'Gossiping', corp = '王品集團', search = i, start_page = 1, end_page = 1)

已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_王品牛排.json 輸出成功
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_陶板屋.json 輸出成功
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_藝奇.json 輸出成功
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_夏慕尼.json 輸出成功
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_品田牧場.json 輸出成功
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_青花驕.json 輸出成功
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_享鴨.json 輸出成功
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_和牛涮.json 輸出成功
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_肉次方.json 輸出成功
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_12mini.json 輸出成功


In [ ]:
#西堤
ptt_crawler(board = 'Gossiping', corp = '王品集團', search = '西堤', start_page = 1, end_page = 10)

已經完成 Gossiping 頁面第 1 頁的爬取
已經完成 Gossiping 頁面第 2 頁的爬取
已經完成 Gossiping 頁面第 3 頁的爬取
已經完成 Gossiping 頁面第 4 頁的爬取
已經完成 Gossiping 頁面第 5 頁的爬取
已經完成 Gossiping 頁面第 6 頁的爬取
已經完成 Gossiping 頁面第 7 頁的爬取
已經完成 Gossiping 頁面第 8 頁的爬取
已經完成 Gossiping 頁面第 9 頁的爬取
已經完成 Gossiping 頁面第 10 頁的爬取
爬取完成 ptt_Gossiping_西堤.json 輸出成功


In [ ]:
#石二鍋
ptt_crawler(board = 'Gossiping', corp = '王品集團', search = '石二鍋', start_page = 1, end_page = 1)

已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_石二鍋.json 輸出成功


In [ ]:
#Food
#王品牛排 1頁

ll = ['王品牛排','西堤','陶板屋','原燒','藝奇','夏慕尼','石二鍋','品田牧場','青花驕','享鴨','the wang','丰禾','肉次方','hot7','尬鍋','嚮辣','12mini']
for i in ll:
    ptt_crawler(board = 'Food', corp = '王品集團', search = i, start_page = 1, end_page = 1)

已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_王品牛排.json 輸出成功
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_西堤.json 輸出成功
list index out of range
在分析 https://www.ptt.cc/bbs/Food/M.1599139044.A.EE2.html 時出現錯誤
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_陶板屋.json 輸出成功
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_原燒.json 輸出成功
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_藝奇.json 輸出成功
list index out of range
在分析 https://www.ptt.cc/bbs/Food/M.1598764845.A.B64.html 時出現錯誤
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_夏慕尼.json 輸出成功
list index out of range
在分析 https://www.ptt.cc/bbs/Food/M.1599055881.A.3C7.html 時出現錯誤
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_石二鍋.json 輸出成功
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_品田牧場.json 輸出成功
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_青花驕.json 輸出成功
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_享鴨.json 輸出成功
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_the wang.json 輸出成功
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_丰禾.json 輸出成功
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_肉次方.json 輸出成功
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_hot7.json 輸出成功
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_尬鍋.json 輸出成功
已經完成 

In [ ]:
timeString = "Fri May 13 13:46:57 2022"
struct_time = time.strptime(timeString, "%a %b %d %H:%M:%S %Y")
new_timeString = time.strftime("%Y-%m-%d %H:%M:%S", struct_time)
new_timeString

'2022-05-13 13:46:57'

In [12]:
import json
import pandas as pd
import requests
import time
import sys, urllib
from bs4 import BeautifulSoup
from bs4.element import NavigableString


root = "https://www.ptt.cc/bbs/"
main = "https://www.ptt.cc"
gossip_data = {
        "from":"bbs/Gossiping/index.html",
        "yes": "yes"
    }

rs = requests.session()

# 爬文章網址
def articles(page):
    requests.packages.urllib3.disable_warnings()
    res = rs.post('https://www.ptt.cc/ask/over18', verify=False, data=gossip_data)
    res  = rs.get(page)
    soup = BeautifulSoup(res.text, "lxml")

    for article in soup.select(".r-ent"):
        try:
            yield main + article.select(".title")[0].select("a")[0].get("href")
        except:
            pass # (本文已被刪除)

# 轉換時間格式
def timechange(day):
    timeString = day
    struct_time = time.strptime(timeString, "%a %b %d %H:%M:%S %Y")
    new_timeString = time.strftime("%Y-%m-%d", struct_time)
    return new_timeString

# 轉換推文的時間
def timechange_push(year, day):
    timeString = year + day.replace('\n', '')
    struct_time = time.strptime(timeString, "%Y %m/%d %H:%M")
    new_timeString = time.strftime("%Y-%m-%d", struct_time)
    return new_timeString

# 爬內文    
def parse_article(url, board = 'Food', corp = '王品集團', Brand = "饗饗"):       
    raw  = rs.get(url)
    soup = BeautifulSoup(raw.text, "lxml")

    try:
        article = {}

        # 取得文章作者與文章標題
        article["Corp"]   = corp
        article["Brand"]  = Brand
        article["Platform"]  = "Ptt"
        article["Branch"]  = ""
        article["Username"] = soup.select(".article-meta-value")[0].contents[0].split(" ")[0]
        article["ReviewTime"]   = timechange(soup.select(".article-meta-value")[3].contents[0])
        article["Title"]  = soup.select(".article-meta-value")[2].contents[0]
        

        # 取得內文
        content = ""
        for t in soup.select("#main-content")[0]:
            if type(t) is NavigableString and t !='\n':
                content += t
                break
        article["ReviewContent"] = content
        article["ReviewStar"] = ""
        article["commentCount"] = ""
        

        # 處理回文資訊
        
        year = soup.select(".article-meta-value")[3].contents[0][-4:]
        article_response_list = []
        
        for response_struct in soup.select(".push"):
            # 跳脫「檔案過大！部分文章無法顯示」
            if "warning-box" not in response_struct['class']:

                article_response = {}
                article_response["Corp"]   = corp
                article_response["Brand"]  = Brand
                article_response["Platform"]  = "Ptt"
                article_response["Branch"]  = ""
                article_response["Username"] = response_struct.select(".push-userid")[0].contents[0] 
                if board != 'Gossiping':
                    article_response["ReviewTime"] = timechange_push(year, response_struct.select(".push-ipdatetime")[0].contents[0])
                else:
                    article_response["ReviewTime"] = timechange_push(year, response_struct.select(".push-ipdatetime")[0].contents[0][-13:])
                article_response["Title"]  = soup.select(".article-meta-value")[2].contents[0]
                article_response["ReviewContent"]= response_struct.select(".push-content")[0].contents[0][1:]
                article_response["ReviewStar"] = ""
                article_response["commentCount"] = ""
                article_response_list.append(article_response)

    except Exception as e:
        print(e)
        print(u"在分析 %s 時出現錯誤" % url)
    
    final_art = [article]
    
    for i in article_response_list:
        final_art.append(i)

    return final_art

def output(filename, data):
    try:
        with open(filename +".json", 'wb+') as f:
            f.write(json.dumps(data, indent=1, ensure_ascii=False).encode('utf-8'))
            print('爬取完成', filename + '.json', '輸出成功')
            
            with open(filename +".json", 'r', encoding = "utf-8") as f:
              a = json.loads(f.read())

            df = pd.json_normalize(a)
            df.to_csv(filename +'.csv', encoding='utf-8-sig', index=False)

    except Exception as err:
        print(filename +'.json', '輸出失敗')
        print('error message:', err) 

def ptt_crawler(board = 'Gossiping', corp = '王品集團', search = '%E7%8E%8B%E5%93%81', start_page = 1, end_page = 2):
    res = []
    for j in range(start_page, end_page+1):
        url = 'https://www.ptt.cc/bbs/'+ board + '/search?page='+ str(j) + '&q=' + search
        for i in articles(url):
            res.extend(parse_article(i, board = board, corp = corp, Brand = search))
            time.sleep(1)   
        print(u"已經完成 %s 頁面第 %d 頁的爬取" %(board, j))

    output('ptt_'+ board+ '_'+ search, res) 

    
#饗饗 %E9%A5%97%E9%A5%97
# ptt_crawler(board = 'Gossiping', search = '%E9%A5%97%E9%A5%97', start_page = 1, end_page = 4)


In [2]:
year = soup.select(".article-meta-value")[3].contents[0][-4:]
timeString = year+'05/24 19:39'
struct_time = time.strptime(timeString, "%Y%m/%d %H:%M")
new_timeString = time.strftime("%Y-%m-%d", struct_time)
print(new_timeString)

NameError: ignored

In [3]:
A = []
A.extend([{'a':1},{'b':2}])
print(A)

[{'a': 1}, {'b': 2}]


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
#Gossip
#饗饗 2頁
ptt_crawler(board = 'Gossiping', corp = '饗賓集團', search = '饗饗', start_page = 1, end_page = 2)

已經完成 Gossiping 頁面第 1 頁的爬取
time data '202107/22 16:41' does not match format '%Y %m/%d %H:%M'
在分析 https://www.ptt.cc/bbs/Gossiping/M.1621780151.A.AA4.html 時出現錯誤
已經完成 Gossiping 頁面第 2 頁的爬取
爬取完成 ptt_Gossiping_饗饗.json 輸出成功


In [14]:
#Gossip
#饗食天堂 2頁
ptt_crawler(board = 'Gossiping', corp = '饗賓集團', search = '饗食天堂', start_page = 1, end_page = 2)

已經完成 Gossiping 頁面第 1 頁的爬取
已經完成 Gossiping 頁面第 2 頁的爬取
爬取完成 ptt_Gossiping_饗食天堂.json 輸出成功


In [15]:
#Gossip
#旭集 2頁
ptt_crawler(board = 'Gossiping', corp = '饗賓集團', search = '旭集', start_page = 1, end_page = 2)

已經完成 Gossiping 頁面第 1 頁的爬取
已經完成 Gossiping 頁面第 2 頁的爬取
爬取完成 ptt_Gossiping_旭集.json 輸出成功


In [16]:
#Gossip
#開飯川 1頁
ptt_crawler(board = 'Gossiping', corp = '饗賓集團', search = '開飯川', start_page = 1, end_page = 1)

已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_開飯川.json 輸出成功


In [17]:
#Gossip
#果然匯 1頁
ptt_crawler(board = 'Gossiping', corp = '饗賓集團', search = '果然匯', start_page = 1, end_page = 1)

已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_果然匯.json 輸出成功


In [18]:
#Food
#饗饗  1頁
ptt_crawler(board = 'Food', corp = '饗賓集團', search = '饗饗', start_page = 1, end_page = 1)

time data '202005/08 13:55' does not match format '%Y %m/%d %H:%M'
在分析 https://www.ptt.cc/bbs/Food/M.1588909217.A.184.html 時出現錯誤
list index out of range
在分析 https://www.ptt.cc/bbs/Food/M.1569208484.A.6B1.html 時出現錯誤


Exception ignored in: <generator object articles at 0x7f5d55b5ae50>
RuntimeError: generator ignored GeneratorExit


UnboundLocalError: ignored

In [19]:
#Food
#饗食天堂 1頁
ptt_crawler(board = 'Food', corp = '饗賓集團', search = '饗食天堂', start_page = 1, end_page = 1)

time data '202004/20 12:21' does not match format '%Y %m/%d %H:%M'
在分析 https://www.ptt.cc/bbs/Food/M.1587354838.A.912.html 時出現錯誤
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_饗食天堂.json 輸出成功


In [20]:
#Food
#旭集 1頁
ptt_crawler(board = 'Food', corp = '饗賓集團', search = '旭集', start_page = 1, end_page = 1)

已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_旭集.json 輸出成功


In [21]:
#Food
#開飯川 1頁
ptt_crawler(board = 'Food', corp = '饗賓集團', search = '開飯川', start_page = 1, end_page = 1)

已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_開飯川.json 輸出成功


In [22]:
#Food
#果然匯 1頁
ptt_crawler(board = 'Food', corp = '饗賓集團', search = '果然匯', start_page = 1, end_page = 1)

已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_果然匯.json 輸出成功


In [23]:
#Food
#饗泰多 1頁
ptt_crawler(board = 'Food', corp = '饗賓集團', search = '饗泰多', start_page = 1, end_page = 1)

已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_饗泰多.json 輸出成功


In [24]:
#Gossip
#王品牛排 1頁
# ll = ['王品牛排','西堤','陶板屋','原燒','藝奇','夏慕尼','石二鍋','品田牧場','青花驕','享鴨','the wang','和牛涮','丰禾','肉次方','hot7','尬鍋','最肉','嚮辣','12mini']
ll = ['王品牛排','陶板屋','藝奇','夏慕尼','品田牧場','青花驕','享鴨','和牛涮','肉次方','12mini']
for i in ll:
    ptt_crawler(board = 'Gossiping', corp = '王品集團', search = i, start_page = 1, end_page = 1)

已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_王品牛排.json 輸出成功
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_陶板屋.json 輸出成功
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_藝奇.json 輸出成功
time data '202109/12 14:49' does not match format '%Y %m/%d %H:%M'
在分析 https://www.ptt.cc/bbs/Gossiping/M.1638098731.A.EA2.html 時出現錯誤
time data '202106/15 04:23' does not match format '%Y %m/%d %H:%M'
在分析 https://www.ptt.cc/bbs/Gossiping/M.1633675869.A.AD0.html 時出現錯誤
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_夏慕尼.json 輸出成功
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_品田牧場.json 輸出成功
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_青花驕.json 輸出成功
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_享鴨.json 輸出成功
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_和牛涮.json 輸出成功
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_肉次方.json 輸出成功
ptt_Gossiping_肉次方.json 輸出失敗
error message: Expecting value: line 1 column 1 (char 0)
已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_12mini.json 輸出成功
ptt_Gossiping_12mini.json 輸出失敗
error message: Expectin

In [25]:
#西堤
ptt_crawler(board = 'Gossiping', corp = '王品集團', search = '西堤', start_page = 1, end_page = 10)

已經完成 Gossiping 頁面第 1 頁的爬取
time data '202205/30 00:43' does not match format '%Y %m/%d %H:%M'
在分析 https://www.ptt.cc/bbs/Gossiping/M.1642817059.A.824.html 時出現錯誤
已經完成 Gossiping 頁面第 2 頁的爬取
已經完成 Gossiping 頁面第 3 頁的爬取
已經完成 Gossiping 頁面第 4 頁的爬取
time data '202212/15 16:25' does not match format '%Y %m/%d %H:%M'
在分析 https://www.ptt.cc/bbs/Gossiping/M.1642413285.A.C60.html 時出現錯誤
已經完成 Gossiping 頁面第 5 頁的爬取
已經完成 Gossiping 頁面第 6 頁的爬取
已經完成 Gossiping 頁面第 7 頁的爬取
已經完成 Gossiping 頁面第 8 頁的爬取
已經完成 Gossiping 頁面第 9 頁的爬取
已經完成 Gossiping 頁面第 10 頁的爬取
爬取完成 ptt_Gossiping_西堤.json 輸出成功


In [26]:
#石二鍋
ptt_crawler(board = 'Gossiping', corp = '王品集團', search = '石二鍋', start_page = 1, end_page = 1)

已經完成 Gossiping 頁面第 1 頁的爬取
爬取完成 ptt_Gossiping_石二鍋.json 輸出成功


In [27]:
#Food
#王品牛排 1頁

ll = ['王品牛排','西堤','陶板屋','原燒','藝奇','夏慕尼','石二鍋','品田牧場','青花驕','享鴨','the wang','丰禾','肉次方','hot7','尬鍋','嚮辣','12mini']
for i in ll:
    ptt_crawler(board = 'Food', corp = '王品集團', search = i, start_page = 1, end_page = 1)

time data '202112/09 13:48' does not match format '%Y %m/%d %H:%M'
在分析 https://www.ptt.cc/bbs/Food/M.1639027673.A.943.html 時出現錯誤
time data '202103/23 14:22' does not match format '%Y %m/%d %H:%M'
在分析 https://www.ptt.cc/bbs/Food/M.1616479158.A.A8A.html 時出現錯誤
time data '202005/19 13:24' does not match format '%Y %m/%d %H:%M'
在分析 https://www.ptt.cc/bbs/Food/M.1589860862.A.62B.html 時出現錯誤
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_王品牛排.json 輸出成功
已經完成 Food 頁面第 1 頁的爬取
爬取完成 ptt_Food_西堤.json 輸出成功
list index out of range
在分析 https://www.ptt.cc/bbs/Food/M.1599139044.A.EE2.html 時出現錯誤


Exception ignored in: <generator object articles at 0x7f5d55f364d0>
RuntimeError: generator ignored GeneratorExit


UnboundLocalError: ignored